In [1]:
import os

In [2]:
#may need to specify a starting path as ARC defualts the directory to whatever folder the Arc file is in, not this script
#os.chdir(path to scripts directory)

In [2]:
os.chdir(r'D:\Documents\OwnershipMap\New_Script\Test_ENV\SCRIPTS')

In [3]:
arcpy.env.workspace = os.path.dirname(os.getcwd())+'\\INPUTS\\State_GDBs'

In [5]:
#for loop through everything in the State_GDBs folder and export to intermidiate folder

In [4]:
for gdb in arcpy.ListFiles():
    name = gdb.split('.')[0]
    arcpy.env.workspace = arcpy.env.workspace+'\\'+ name+'.gdb'
    arcpy.CreateFolder_management(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\', name)
    outpath = os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE'+'\\'+name
    parcels = [i for i in arcpy.ListFeatureClasses() if i.startswith('Par')][0]
    points = [i for i in arcpy.ListFeatureClasses() if i.startswith('Prop')][0]
    arcpy.FeaturesToJSON_conversion(parcels, outpath+'\\'+name+"_parcel.json", "FORMATTED", "", "", "GEOJSON", "WGS84")
    arcpy.FeaturesToJSON_conversion(points, outpath+'\\'+name+"_points.json", "FORMATTED", "", "", "GEOJSON", "WGS84")
    arcpy.conversion.PolygonToRaster(in_features=arcpy.env.workspace +'\\'+parcels, 
                                     value_field="PRCLDMPID", 
                                     out_rasterdataset=outpath+'\\'+name+"_Parcel_IDs.tif", 
                                     cell_assignment="CELL_CENTER", 
                                     priority_field="NONE", 
                                     cellsize = os.path.dirname(os.getcwd())+'\\INPUTS\\Reference_Raster.tif', 
                                     build_rat="BUILD")
    arcpy.TableToTable_conversion(outpath+'\\'+name+"_Parcel_IDs.tif", os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\'+name, "ID_Dictionary.csv")
    arcpy.management.Clip(os.path.dirname(os.getcwd())+'//INPUTS//NLCD_10m_NAD83.tif',
                          "",
                          outpath+'//'+name+"_NLCD.tif",
                          parcels,
                          "255", 
                          "NONE", 
                          "NO_MAINTAIN_EXTENT")
    remap = arcpy.sa.Reclassify(outpath+'//'+name+"_NLCD.tif", 
                           "NLCD_Land", 
                           "Unclassified -99999;'Open Water' 100000000;'Developed, Open Space' 200000000;'Developed, Low Intensity' 300000000;'Developed, Medium Intensity' 400000000;'Developed, High Intensity' 500000000;'Barren Land' 600000000;'Deciduous Forest' 700000000;'Evergreen Forest' 800000000;'Mixed Forest' 900000000;Shrub/Scrub 110000000;Herbaceuous 120000000;Hay/Pasture 130000000;'Cultivated Crops' 140000000;'Woody Wetlands' 150000000;'Emergent Herbaceuous Wetlands' 160000000", 
                           0)
    #remap = arcpy.sa.Reclassify(outpath+'//'+name+"_NLCD.tif", 
                           #"NLCD_Land", 
                           #"Unclassified -99999;'Open Water' 100000000;'Developed, Open Space' 200000000;'Developed, Low Intensity' 300000000;'Developed, Medium Intensity' 400000000;'Developed, High Intensity' 500000000;'Barren Land' 600000000;'Deciduous Forest' 700000000;'Evergreen Forest' 800000000;'Mixed Forest' 900000000;Shrub/Scrub 110000000;Herbaceuous 120000000;Hay/Pasture 130000000;'Cultivated Crops' 140000000;'Woody Wetlands' 150000000;'Emergent Herbaceuous Wetlands' 160000000", 
                           #"NODATA")
    (outpath+'\\'+name+"_Parcel_IDs.tif" + remap).save(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\'+name+'\\ID_NLCD_temp.tif')
    arcpy.env.workspace = os.path.dirname(os.getcwd())+'\\INPUTS\\State_GDBs'

TypeError: unsupported operand type(s) for -: 'builtin_function_or_method' and 'float'